<a href="https://colab.research.google.com/github/TurkuNLP/intro-to-nlp/blob/master/w2v_gensim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gensim

* A useful library specifically for dealing with static word embeddings: their training and loading
* https://radimrehurek.com/gensim/auto_examples/index.html#documentation
* For our purposes, this is really easy to use

# Embedding repositories

* Gensim's own: https://github.com/RaRe-Technologies/gensim-data
* NLPL vectors: http://vectors.nlpl.eu/repository/

In [1]:
import gensim

In [2]:
# I found this link in the NLPL repository
# It refers to English model trained on the Gigaword corpus of news

!wget http://vectors.nlpl.eu/repository/20/12.zip
!unzip 12.zip

--2023-04-11 08:21:04--  http://vectors.nlpl.eu/repository/20/12.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.181
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 613577258 (585M) [application/zip]
Saving to: ‘12.zip’

12.zip              100%[===================>] 585.15M  24.9MB/s    in 25s     

2023-04-11 08:21:30 (23.8 MB/s) - ‘12.zip’ saved [613577258/613577258]

Archive:  12.zip
  inflating: meta.json               
  inflating: model.bin               
  inflating: model.txt               
  inflating: README                  


In [13]:
# This is how you load the trained embeddings

from gensim.models import KeyedVectors

wv_emb=KeyedVectors.load_word2vec_format("model.bin", binary=True)

`KeyedVectors` documentation is here: https://radimrehurek.com/gensim/models/keyedvectors.html

In [16]:
result = wv_emb.similar_by_word('cat')
for most_similar_key, similarity in result[:10]:
    print(f"{most_similar_key}: {similarity:.4f}")

dog: 0.8098
cats: 0.7977
mutt: 0.7505
kitten: 0.7471
feline: 0.7438
puppy: 0.7252
felines: 0.7251
raccoon: 0.7183
pooch: 0.7116
dachshund: 0.7018
